In [ ]:
# Load library imports
import os
import sys
import torch
import joblib
import random
import logging
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd

# Load project Imports
from src.utils.config_loader import load_project_config, deep_format, expanduser_tree
from src.graph_building.graph_construction import build_mesh, \
    define_catchment_polygon, define_graph_adjacency
from src.preprocessing.data_partitioning import define_station_id_splits, \
    load_graph_tensors, build_pyg_object
from src.preprocessing.model_feature_engineering import preprocess_gwl_features, \
    preprocess_shared_features

In [ ]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up root directory paths in config
raw_data_root = config["global"]["paths"]["raw_data_root"]
results_root = config["global"]["paths"]["results_root"]

# Reformat config roots
config = deep_format(
    config,
    raw_data_root=raw_data_root,
    results_root=results_root
)
config = expanduser_tree(config)

In [ ]:
# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

In [ ]:
# Select Catchment area from country wide gdf
define_catchment_polygon(
    england_catchment_gdf_path=config[catchment]['paths']['gis_catchment_boundary'],
    target_mncat=config[catchment]['target_mncat'],
    catchment=catchment,
    polygon_output_path=config[catchment]['paths']['gis_catchment_dir']
)

# Build catchment mesh
mesh_nodes_table, mesh_nodes_gdf, mesh_cells_gdf_polygons, catchment_polygon = build_mesh(
    shape_filepath=config[catchment]['paths']['gis_catchment_dir'],
    output_path=config[catchment]['paths']['mesh_nodes_output'],
    catchment=catchment,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']
)

logger.info(f"Pipeline step 'Build Mesh' complete for {catchment} catchment.")

In [ ]:
directional_edge_path = config[catchment]["paths"]["direction_edge_weights_path"]
directional_edge_weights = pd.read_csv(directional_edge_path)

# Create specific node_id column to merge
directional_edge_weights["node_id"] = range(0, len(directional_edge_weights))
directional_edge_weights

In [ ]:
mesh_cells_gdf_polygons

### BUILD EDGE WEIGHTS ###

In [ ]:
# Load in directional edge weights and mean elevation (not req. in main pipeline)
directional_edge_path=config[catchment]["paths"]["direction_edge_weights_path"]
directional_edge_weights = pd.read_csv(directional_edge_path)

edge_attr_tensor, edge_index_tensor = define_graph_adjacency(
    directional_edge_weights=directional_edge_weights,
    elevation_geojson_path=config[catchment]['paths']['elevation_geojson_path'],
    graph_output_dir=config[catchment]["paths"]["graph_data_output_dir"],
    mesh_cells_gdf_polygons=mesh_cells_gdf_polygons,
    epsilon_path=config["global"]["graph"]["epsilon"],
    catchment=catchment
)

logger.info(f"Pipeline step 'Define Graph Adjacency' complete for {catchment} catchment.\n")

In [ ]:
edge_attr_tensor

In [ ]:
edge_index_tensor

### Split into Train/Val/Test Split ###

In [ ]:
# # Load tensors from file if needed
# edge_index_tensor, edge_attr_tensor = load_graph_tensors(
#     graph_output_dir=config[catchment]["paths"]["graph_data_output_dir"],
#     catchment=catchment
# )

# Load main_df_full from file if needed
load_path = config[catchment]["paths"]["final_df_path"] + 'final_df.csv'
main_df_full = pd.read_csv(load_path)
main_df_full

In [ ]:
# --- 6a. Define Spatial Split for Observed Stations ---

train_station_ids, val_station_ids, test_station_ids = define_station_id_splits(
    main_df_full=main_df_full,
    catchment=catchment,
    test_station_shortlist=config[catchment]["model"]["data_partioning"]["test_station_shortlist"],
    val_station_shortlist=config[catchment]["model"]["data_partioning"]["val_station_shortlist"],
    random_seed=config["global"]["pipeline_settings"]["random_seed"],
    output_dir=config[catchment]["paths"]["aux_dir"],
    perc_train=config[catchment]["model"]["data_partioning"]["percentage_train"],
    perc_val=config[catchment]["model"]["data_partioning"]["percentage_val"],
    perc_test=config[catchment]["model"]["data_partioning"]["percentage_test"]
)

logger.info(f"Pipeline Step 'define station splits' complete for {catchment} catchment.")

### Preprocess Shared Features Prior to Split

In [ ]:
# --- 6b. Preprocess (Standardise, one hot encode, round to 4dp) all shared features (not GWL) ---

processed_df, shared_scaler, shared_encoder, gwl_feats = preprocess_shared_features(
    main_df_full=main_df_full,
    catchment=catchment,
    random_seed=config["global"]["pipeline_settings"]["random_seed"],
    violin_plt_path=config[catchment]["visualisations"]["violin_plt_path"],
    scaler_dir = config[catchment]["paths"]["scalers_dir"],
    aux_dir=config[catchment]["paths"]["aux_dir"]
)

logger.info(f"Pipeline Step 'Preprocess Final Shared Features' complete for {catchment} catchment.")

### Split processed_df into train/val/test subsets

In [ ]:
# --- 6c. Preprocess all GWL features using training data only ---

processed_df, gwl_scaler, gwl_encoder = preprocess_gwl_features(
    processed_df=processed_df,
    catchment=catchment,
    train_station_ids=train_station_ids,
    val_station_ids=val_station_ids,
    test_station_ids=test_station_ids,
    sentinel_value=config["global"]["graph"]["sentinel_value"],
    scaler_dir=config[catchment]["paths"]["scalers_dir"],
    parquet_path=os.path.join(config[catchment]["paths"]["final_df_path"], 'processed_df.parquet')
)

logger.info(f"Pipeline Step 'Preprocess Final GWL Features' complete for {catchment} catchment.")

In [ ]:
print("Columns in processed_df after preprocessing:", processed_df.columns)

In [ ]:
# # Shorten df to test range to reduce computation requirements

# test_start_date_str = config["global"]["data_ingestion"]["test_start_date"]
# test_end_date_str = config["global"]["data_ingestion"]["test_end_date"]

# # Convert 'timestep' column to datetime objects
# processed_df['timestep'] = pd.to_datetime(processed_df['timestep'])

# processed_df_test = processed_df.loc[
#     (processed_df['timestep'] >= test_start_date_str) &
#     (processed_df['timestep'] <= test_end_date_str)
# ].copy()

# print(f"Original processed_df shape (full data): {processed_df.shape}")
# print(f"Test processed_df_test shape (sliced from {test_start_date_str} to {test_end_date_str}): {processed_df_test.shape}")
processed_df_test = processed_df.drop(columns=['streamflow_total_m3', 'HOST_soil_class_freely_draining_soils', 'HOST_soil_class_high_runoff_(impermeable)', 
                                               'HOST_soil_class_impeded_saturated_subsurface_flow', 'HOST_soil_class_peat_soils', 'aquifer_productivity_High',
                                               'aquifer_productivity_Low', 'aquifer_productivity_Mixed', 'aquifer_productivity_Moderate',
                                               'aquifer_productivity_nan']).copy()

In [ ]:
# Display full (test) processed df
processed_df_test

In [ ]:
processed_df_test.describe()

### Create Train/Val/Test PyG Objects for Model Input

In [ ]:
# --- 6d. Create PyG data objects using partioned station IDs (from 6a) ---

# Run time approx. 13.5 mins to build 4018 timesteps of objects (0.201s per timestep)
gwl_ohe_cols = joblib.load(os.path.join(config[catchment]["paths"]["scalers_dir"], "gwl_ohe_cols.pkl"))
all_timesteps_list = build_pyg_object(
    processed_df=processed_df_test,
    sentinel_value=config["global"]["graph"]["sentinel_value"],
    train_station_ids=train_station_ids,
    val_station_ids=val_station_ids,
    test_station_ids=test_station_ids,
    gwl_feats=gwl_feats,
    gwl_ohe_cols=gwl_ohe_cols,
    edge_index_tensor=edge_index_tensor,
    edge_attr_tensor=edge_attr_tensor,
    scalers_dir=config[catchment]["paths"]["scalers_dir"],
    catchment=catchment
)

# Save all_timesteps_list to file
torch.save(all_timesteps_list, config[catchment]["paths"]["pyg_object_path"])
logger.info(f"Pipeline Step 'Build PyG Data Objects' complete for {catchment} catchment.")